In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

In [ ]:
movies = pd.read_csv('../input/movielens-latest-small/movies.csv')


#data preprocessing, cleaning
movies['genres'] = movies['genres'].apply(lambda x: x.replace('|', ' '))
movies['name'] = movies['title'].apply(lambda x: x[:-7])

In [ ]:
count = CountVectorizer()
count_matrix = count.fit_transform(movies['genres'])

similarities = cosine_similarity(count_matrix, count_matrix)

In [ ]:
movie_ids = pd.Series(movies.index)

def find_recommendations(name):
    
    recommendations = []    
    movie_id = movies.loc[movies['name'] == name].index[0]

    current_movie_similarities = pd.Series(similarities[movie_id]).sort_values(ascending = False)

    top_ten = list(current_movie_similarities.iloc[:11].index)
    
    top_ten.remove(movie_id)
    
    for movie in top_ten:
        recommendations.append(list(movies.index)[movie])
        
    for movie in recommendations:
        print(movies.iloc[movie]['title'])

In [ ]:
find_recommendations('Up')

In [ ]:
ratings = pd.read_csv('../input/movielens-latest-small/ratings.csv')

In [ ]:
#ratings_data = pd.merge(movies, ratings, on='movieId')
#ratings_data[(ratings_data['userId'] == 70) & (ratings_data['rating'] >= 4.0)]

In [ ]:
import surprise

reader = surprise.Reader(rating_scale=(0.5, 5.))
data = surprise.Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

In [ ]:
model = surprise.SVDpp()
fitted_model = model.fit(data.build_full_trainset())

In [ ]:
def find_collaborative_recommendations(user_id):
    movies_watched_by_user = ratings.loc[ratings['userId'] == user_id, 'movieId']
    unwatched_movies = np.setdiff1d(movie_ids, movies_watched_by_user)
    
    user_set = [[user_id, movie_id, 4.0] for movie_id in unwatched_movies]
    predictions = fitted_model.test(user_set)
    predicted_ratings = np.array([prediction.est for prediction in predictions])
    top_n = 10
    max_rating_indexes = (-predicted_ratings).argsort()[:top_n]
    for movie in max_rating_indexes:
        movie_id = unwatched_movies[movie]
        print(movies.iloc[movie_id]['title'])

In [ ]:
find_collaborative_recommendations(70)